<a href="https://colab.research.google.com/github/gmestrallet/IntegratingSpatialAndScRNAseq/blob/main/IntegratingSpatialAndScRNAseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to integrate visium spatial and single cell RNAseq data using scanpy with data stored in google drive. More information available https://scanpy.readthedocs.io/en/stable/tutorials/spatial/integration-scanorama.html

In [ ]:
#Mount Google Drive to access your files, if they are stored there.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Set the path where you want to store the files (use your own directory).
import os

In [ ]:
#Replace 'RNAseq_folder' with the path to the folder in your Google Drive or use '/content/' for local storage.
rna_seq_path = '/content/drive/My Drive/RNAseq_folder'
os.chdir(rna_seq_path)

In [ ]:
#Install necessary libraries and import
!pip install scanpy  # Make sure scanpy is installed
!pip install anndata  # Make sure anndata is installed
!pip install scanorama  # Make sure scanorama is installed
!pip install h5py  # Make sure h5py is installed
!pip install cupy-cuda11x  # Make sure cupy is installed (change if using a different CUDA version)
!pip install leidenalg  # Make sure leidenalg is installed
import scanpy as sc
import anndata as an
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanorama
import h5py
import cupy

In [ ]:
sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3

In [ ]:
#Read the filtered feature barcode matrix (gene expression data such as filtered_feature_bc_matrix.h5)
adata = sc.read_visium("/content/drive/My Drive/RNAseq_folder")
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

In [ ]:
adata

In [ ]:
#QC and preprocessing, we perform basic filtering of spots based on total counts and expressed genes
fig, axs = plt.subplots(1, 4, figsize=(15, 4))
sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0])
sns.histplot(
    adata.obs["total_counts"][adata.obs["total_counts"] < 10000],
    kde=False,
    bins=40,
    ax=axs[1],
)
sns.histplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2])
sns.histplot(
    adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000],
    kde=False,
    bins=60,
    ax=axs[3],
)

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000, inplace=True)

In [ ]:
#Load preprocessed and clustered scRNAseq data if this is already done (it should be in your previous write directory, see https://colab.research.google.com/drive/1qOinOkYVgRqIetGPbvx7F6mzDVUIlws8#scrollTo=PZ4UcjrcMfiw)
single_cells = sc.read('/content/drive/My Drive/RNAseq_folder/data/write/allcells.h5ad')

In [ ]:
single_cells

In [ ]:
adatas = [single_cells, adata]

# Integration.
adatas_cor = scanorama.correct_scanpy(adatas, return_dimred=True)

In [ ]:
adata2 = sc.concat(
    adatas_cor,
    label="dataset",
    keys=["smart-seq", "visium"],
    join="outer",
    uns_merge="first",
)

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
distances = 1 - cosine_distances(
    adata2[adata2.obs.dataset == "smart-seq"].obsm["X_scanorama"],
    adata2[adata2.obs.dataset == "visium"].obsm["X_scanorama"],
)

In [ ]:
def label_transfer(dist, labels):
    lab = pd.get_dummies(labels).to_numpy().T
    class_prob = lab @ dist
    norm = np.linalg.norm(class_prob, 2, axis=0)
    class_prob = class_prob / norm
    class_prob = (class_prob.T - class_prob.min(1)) / class_prob.ptp(1)
    return class_prob

In [ ]:
class_prob = label_transfer(distances, single_cells.obs['leiden'])

In [ ]:
cp_df = pd.DataFrame(
    class_prob, columns=np.sort(single_cells.obs['leiden'].unique())
)
cp_df.index = adata.obs.index

In [ ]:
adata_transfer = adata.copy()
adata_transfer.obs = pd.concat(
    [adata.obs, cp_df], axis=1
)

In [ ]:
adata_transfer.obsm['spatial'] = adata_transfer.obsm['spatial'].astype(int)
sc.pl.spatial(
    adata_transfer,
    img_key="hires",
    color=['ScRNASeq_Cluster_1','ScRNASeq_Cluster_2'],
    size=1.5,
)